# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнила: Анастасия Успенская

### Общая информация

__Дата выдачи:__ 06.04.2020

__Дедлайн:__ 23:59 20.04.2020


### Оценивание и штрафы

Оценка за ДЗ вычисляется по следующей формуле:

$$
min(\text{points}, 18)  \times 10 / 18,
$$

где points — количество баллов за обязательную часть, которое вы набрали. Максимальное число баллов, которое можно получить за обязательную часть — 18, за каждые полтора балла сверху вы получите 1 бонусный балл (максимум 2). Также вы можете использовать бонусные баллы, которые накопили ранее.

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи

Загрузка файлов с решениями происходит в системе [Anytask](https://anytask.org/).

Инвайт для группы ИАД-6: rd5CNrr

Перед отправкой перезагрузите ноутбук и проверьте, что все ячейки могут быть последовательно выполнены. Ноутбук должен запускаться с использованием python 3.6+

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [ ]:
# Качаем датасет

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [2]:
data = pd.read_csv("https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv?dl=1", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [ ]:
data.shape

In [3]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train[:5]

In [ ]:
y_train[:5]

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Можете использовать WordPunctTokenizer или подобрать какой-то другой, если считаете, что он лучше подойдет для этой задачи.


In [5]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'

print("before:", text,)
print("after:", tokenizer.tokenize(text.lower()),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: ['здраствуйте', '.', 'я', ',', 'кирилл', '.', 'хотел', 'бы', 'чтобы', 'вы', 'сделали', 'игру', ',', '3д', '-', 'экшон', 'суть', 'такова', '...']


__Задание:__ реализуйте функцию ниже

In [6]:
def preprocess(text: str) -> str:
    """
    Данная функция принимает на вход текст, 
    а возвращает тот же текст, но с пробелами между каждым токеном
    """
    return ' '.join(tokenizer.tokenize(text.lower()))
preprocess(text)

'здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...'

In [7]:
assert preprocess(text) == 'здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...'

__Задание:__ токенизируйте title и description в train и test

In [8]:
from itertools import chain
for x in chain(X_train, X_test):
    x[0] = preprocess(x[0])
    x[1] = preprocess(x[1])

In [9]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

In [ ]:
X_test[10][1]

### BOW (3 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ создайте словарь, где в соответствии каждому токену стоит количество раз, которое оно встретилось в X_train

In [ ]:
X_train[:5]

In [10]:
from itertools import chain
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
senlist = [j for sub in X_train for j in sub]

In [11]:
from itertools import dropwhile
import re
for i in range(len(senlist)):
    senlist [i]= re.sub(r'\W|\b\d+\b',' ', senlist [i]) # Убираем пунктуацию и цифры, но оставляем вещи типа 128gb
    senlist [i] = re.sub(r'\s+',' ', senlist [i])

In [ ]:
senlist[10]

In [12]:
tokens = {}
for i in senlist:
    freq = word_tokenize(i)
    for token in freq:
        if token not in tokens.keys():
            tokens[token] = 1
        else:
            tokens[token] += 1

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
tokens

{'сапоги': 454,
 'размер': 3320,
 'новые': 1872,
 'светильники': 26,
 'потолочный': 8,
 'swarovski': 9,
 'потолочные': 8,
 'штук': 135,
 'цена': 2379,
 'за': 3425,
 'штуку': 134,
 'в': 28337,
 'эксплуатации': 157,
 'года': 1012,
 'продаются': 152,
 'связи': 322,
 'со': 853,
 'сменой': 10,
 'интерьера': 39,
 'квартире': 261,
 'iphone': 465,
 'plus': 163,
 '128gb': 25,
 'red': 34,
 'красный': 121,
 'наличии': 2172,
 'данная': 149,
 'только': 1129,
 'для': 9627,
 'подписчиков': 2,
 'instagram': 39,
 'iqmac': 1,
 'новый': 1626,
 'айфон': 37,
 'это': 744,
 'элегантный': 14,
 'и': 21714,
 'мощный': 53,
 'смартфон': 70,
 'который': 200,
 'готов': 82,
 'полной': 71,
 'мере': 9,
 'раскрыть': 4,
 'возможности': 70,
 'ios': 37,
 'аппарат': 117,
 'с': 12860,
 'ядерным': 1,
 'процессором': 5,
 'а10': 2,
 'гб': 213,
 'озу': 24,
 'легкостью': 13,
 'решает': 4,
 'самые': 173,
 'ресурсоемкие': 1,
 'задачи': 25,
 'позволяя': 5,
 'наслаждаться': 15,
 'быстродействием': 1,
 'тяжелых': 8,
 'приложений': 4,

In [15]:
assert tokens['сапоги'] == 454

__Задание:__ выведите 10 самых частотных и 10 самых редких токенов

In [16]:
sorted(tokens.items(), key=lambda x:x[1])[:10]

[('iqmac', 1),
 ('ядерным', 1),
 ('ресурсоемкие', 1),
 ('быстродействием', 1),
 ('кинематографическому', 1),
 ('ирис', 1),
 ('саженец', 1),
 ('корень', 1),
 ('зубчаниновка', 1),
 ('боярышник', 1)]

In [17]:
sorted(tokens.items(), key=lambda x:x[1], reverse=True)[:10]

[('в', 28337),
 ('и', 21714),
 ('на', 19465),
 ('с', 12860),
 ('по', 10000),
 ('для', 9627),
 ('не', 6045),
 ('до', 4541),
 ('от', 4482),
 ('состоянии', 3981)]

__Задание:__ оставьте в словаре только топ 10000 самых частотных токенов

In [18]:
tokens=sorted(tokens.items(), key=lambda x:x[1], reverse=True)[:10000]

In [ ]:
for x, y in tokens:
    if re.match(r'.*\d.*', x):
        print(x, y) #Проверка

In [ ]:
tokens[:10]

In [19]:
most_common_words = [x for x, y in tokens]
most_common_words[:15]

['в',
 'и',
 'на',
 'с',
 'по',
 'для',
 'не',
 'до',
 'от',
 'состоянии',
 'у',
 'за',
 'все',
 'размер',
 'без']

In [20]:
word_to_index = {word: index for index, word in enumerate(most_common_words)}

In [ ]:
word_to_index['состоянии']

__Задание:__ реализуйте функцию, которая предложение переводит в вектор из чисел. То есть каждому слову из словаря сопоставляется количество раз, которое оно встретилось в предложении.

In [21]:
def preprocess2(text: str) -> str:
    text = preprocess(text)
    text = re.sub(r'\W|\b\d+\b',' ', text)
    text = re.sub(r'\s+',' ', text)
    return text

In [22]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    input: строка
    output: вектор размерности словаря
    """
    sentence_tokens = word_tokenize(preprocess2(text))
    sent_vec = np.zeros(len(most_common_words))
    for token in sentence_tokens:
        if token in word_to_index:
            sent_vec[word_to_index[token]] += 1
    return sent_vec

In [23]:
vector = text_to_bow('купить чайник купить ВШЭ') # Проверка
for i in range(len(vector)):
    if vector[i]:
        print(vector[i], most_common_words[i])

2.0 купить
1.0 чайник


__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и для каждого текста из description сопоставляет вектор.

In [24]:
from nltk.tokenize import sent_tokenize
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    descriptions = items[:, 1]
    vectors = np.zeros((len(items), len(most_common_words)))
    for i, desc in tqdm(enumerate(descriptions)):
        vectors[i] = text_to_bow(desc)
    return vectors

In [25]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

21000it [00:10, 2002.69it/s]
9000it [00:04, 1882.95it/s]


In [ ]:
X_train_bow

In [26]:
# чтобы видеть проход по итерациям можно использовать библиотеку tqdm
# она работает примерно так
from tqdm import tqdm

for i in tqdm(range(100)):
    pass

100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<?, ?it/s]


In [27]:
from sklearn.metrics import accuracy_score

### Логистическая регрессия и SVC (0.5 балла)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


In [28]:
from sklearn import linear_model
lr = linear_model.LogisticRegression()
lr_model = lr.fit(X_train_bow, y_train)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [29]:
accuracy_score(lr_model.predict(X_test_bow), y_test)

0.704

In [30]:
assert accuracy_score(lr_model.predict(X_test_bow), y_test) > 0.7

In [ ]:
from sklearn.svm import SVC
SVM_classifier = SVC(C=0.01, kernel='linear')
svc_model = SVM_classifier.fit(X_train_bow, y_train)

In [ ]:
X_train_bow[:500,:100].shape

In [ ]:
# не такое долгое https://datascience.stackexchange.com/questions/989/svm-using-scikit-learn-runs-endlessly-and-never-completes-execution
from sklearn.svm import SVC
SVM_classifier = SVC(C=0.01, kernel='linear')
svc_model = SVM_classifier.fit(X_train_bow, y_train)

In [31]:
accuracy_score(svc_model.predict(X_test_bow), y_test)

0.694

In [ ]:
assert accuracy_score(svc_model.predict(X_test_bow), y_test) > 0.68

### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?

In [32]:
def items_to_bow(items: np.array, column: int) -> np.array:
    descriptions = items[:, column]
    vectors = np.zeros((len(items), len(most_common_words)))
    for i, desc in tqdm(enumerate(descriptions)):
        vectors[i] = text_to_bow(desc)
    return vectors

def full_items_to_bow(items: np.array) -> np.array:
    title_features = items_to_bow(items, 0)
    description_features = items_to_bow(items, 1)
    return np.concatenate([title_features, description_features], axis=1)

In [33]:
X_train_full_bow = full_items_to_bow(X_train)
X_test_full_bow = full_items_to_bow(X_test)

21000it [00:06, 3432.51it/s]
21000it [00:12, 1696.32it/s]
9000it [00:01, 5221.51it/s]
9000it [00:04, 1863.30it/s]


In [34]:
from sklearn import linear_model, metrics
lr = linear_model.LogisticRegression()
lr_model = lr.fit(X_train_full_bow, y_train)

metrics.accuracy_score(lr_model.predict(X_test_full_bow), y_test)

# Accuracy увеличился на 0,08

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7776666666666666

Нормализуйте данные (`sklearn.preprocessing.normalize`) перед обучением. Что станет с качеством и почему?

In [35]:
from sklearn import preprocessing

lr_model = lr.fit(preprocessing.normalize(X_train_full_bow), y_train)

metrics.accuracy_score(lr_model.predict(preprocessing.normalize(X_test_full_bow)), y_test)

# Accuracy уменьшился на 0.6. Возможно, это из-за того, что нормализация происходит по всей выборке (а они
# разные в случае train и test, и отношение TP/(TP+FN) уменьшается.

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6998888888888889

### Mystem (1 балл)

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

In [ ]:
pip install git+https://github.com/nlpub/pymystem3

In [ ]:
from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer


In [ ]:
text = "Красивая мама красиво мыла раму"
m = Mystem()
lemmas = m.lemmatize(text)
print(''.join(lemmas))

In [ ]:
lemmas

In [ ]:
lemmatizer = Mystem()
stemtokens = {}
for text in senlist:
    freq = lemmatizer.lemmatize(text)
    for token in freq:
        if token not in stemtokens.keys():
            stemtokens[token] = 1
        else:
            stemtokens[token] += 1
stemtokens=sorted(stemtokens.items(), key=lambda x:x[1], reverse=True)[:10000]   
stem_most_common_words = [x for x, y in stemtokens]
stem_word_to_index = {word: index for index, word in enumerate(stem_most_common_words)}

In [ ]:
def stem_text_to_bow(text: str) -> np.array:
    lemmatizer = Mystem()
    sentence_tokens = lemmatizer.lemmatize(text)
    sent_vec = np.zeros(len(stem_most_common_words))
    for token in sentence_tokens:
        if token in stem_word_to_index:
            sent_vec[stem_word_to_index[token]] += 1
    return sent_vec

In [ ]:
def stem_items_to_bow(items: np.array) -> np.array:
    modified = items[:,0] + " " + items[:, 1]
    vectors = np.zeros((len(items), len(stem_most_common_words)))
    for i, desc in enumerate(modified):
        vectors[i] = stem_text_to_bow(desc)
    return vectors

In [ ]:
train_stem = stem_items_to_bow(X_train)
test_stem=stem_items_to_bow(X_test)

In [73]:
from sklearn import linear_model
lr = linear_model.LogisticRegression()
lr_model = lr.fit(train_stem, y_train)
accuracy_score(lr_model.predict(X_test_bow), y_test)

NameError: name 'train_stem' is not defined

In [ ]:
from sklearn.svm import SVC
SVM_classifier = SVC(C=0.01, kernel='linear')
svc_model = SVM_classifier.fit(train_stem, y_train)
accuracy_score(svc_model.predict(test_stem), y_test)

### TF-IDF (5 баллов)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя. Для простоты посчитайте общий tf-idf для title и description (то есть каждому объекту надо сопоставить вектор, где как документ будет рассматриваться конкатенация title и description).

__Задание:__ составьте словарь, где каждому слову из изначального словаря будет стоять в соответствии количество документов, где это слово встретилось.

In [36]:
word_to_number_documents = {}
for sentence in senlist:
    for word in set(word_tokenize(sentence)):
        if word in word_to_index:
            if word in word_to_number_documents:
                word_to_number_documents[word] += 1
            else:
                word_to_number_documents[word] = 1

__Задание:__ реализуйте функцию, где тексту в соответствие ставится tf-idf вектор. 

In [37]:
def text_to_tfidf(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    input: строка
    output: вектор размерности словаря
    """
    sentence_tokens = word_tokenize(preprocess2(text))
    sent_vec = np.zeros(len(most_common_words))
    for token in sentence_tokens:
        if token in word_to_index:
            tf = 1 / len(sentence_tokens)
            idf = np.log(len(senlist) / word_to_number_documents[token])
            sent_vec[word_to_index[token]] += tf * idf
    return sent_vec

In [38]:
text_to_tfidf('в купить в чайник') # Проверка

array([0.61245232, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ])

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и для каждого объекта сопоставляет вектор tf-idf.

In [ ]:
len(senlist)

In [ ]:
word_to_number_documents['в']

In [ ]:
tf = 1/4
idf = np.log(42000 / 12339)

In [ ]:
tf * idf

In [39]:
def items_to_tfidf(items: np.array) -> np.array:
    """ 
    Для каждого товара возвращает его tfidf вектор
    """
    modified = items[:,0] + " " + items[:, 1]
    vectors = np.zeros((len(items), len(most_common_words)))
    for i, desc in enumerate(modified):
        vectors[i] = text_to_tfidf(desc)
    return vectors

In [40]:
X_train_tfidf = items_to_tfidf(X_train)
X_test_tfidf = items_to_tfidf(X_test)

__Задание:__ обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [41]:
from sklearn import linear_model
lr = linear_model.LogisticRegression()
lr_model = lr.fit(X_train_tfidf, y_train)

accuracy_score(lr_model.predict(X_test_tfidf), y_test)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7206666666666667

In [ ]:
from sklearn.svm import SVC
SVM_classifier = SVC(C=0.01, kernel='linear')
svc_model = SVM_classifier.fit(X_train_tfidf, y_train)

accuracy_score(svc_model.predict(X_test_tfidf), y_test)

### Hashing Vectorizer (1 балл)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [51]:
from sklearn.feature_extraction.text import HashingVectorizer
hash_vec = HashingVectorizer(n_features=10000)
modified = X_train[:,0] + " " + X_train[:, 1]
texts_tokenized = [' '.join([w for w in word_tokenize(t) if w.isalpha()]) for t in modified]
X = hash_vec.fit_transform(texts_tokenized)
X

<21000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 665481 stored elements in Compressed Sparse Row format>

In [52]:
X = X.toarray()

In [53]:
X.shape

(21000, 10000)

In [54]:
modified2 = X_test[:,0] + " " + X_test[:, 1]
texts_tokenized_test = [' '.join([w for w in word_tokenize(t) if w.isalpha()]) for t in modified2]
X_test_hash=hash_vec.transform(texts_tokenized_test)

In [55]:
from sklearn import linear_model
lr = linear_model.LogisticRegression()
lr_model = lr.fit(X, y_train)

accuracy_score(lr_model.predict(X_test_hash), y_test)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7067777777777777

In [ ]:
from sklearn.svm import SVC
SVM_classifier = SVC(C=0.01, kernel='linear')
svc_model = SVM_classifier.fit(X, y_train)

accuracy_score(svc_model.predict(X_test_hash), y_test)

### Word Vectors (3 балла)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [ ]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

In [ ]:
!tar -xzf ru.tar.gz

In [56]:
import gensim
from gensim.models.wrappers import FastText

model = FastText.load_fasttext_format('ru.bin')

unable to import 'smart_open.gcs', disabling that module


In [ ]:
# как мы видим, каждому слову данная модель сопоставляет вектор размерности 300

print(model['привет'].shape)
print(model['привет'])

In [64]:
def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """
    sentence_tokens = word_tokenize(preprocess2(sentence))
    embedding = np.zeros(300)
    for token in sentence_tokens:
        if token in model:
            embedding += model[token]
        
    return embedding

In [65]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

__Задание:__ сделайте все то же, что в предыдущих пунктах -- реализуйте функцию, которая преобразует данные, а затем обучите логистическую регрессию и SVM, оцените качество.

In [67]:
def word_vectors1(items: np.array, column: int) -> np.array:
    descriptions = items[:, column]
    vectors = np.zeros((len(items), 300))
    for i, desc in tqdm(enumerate(descriptions)):
        vectors[i] = sentence_embedding(desc)
    return vectors

def word_vectors2(items: np.array) -> np.array:
    title_features = word_vectors1(items, 0)
    description_features = word_vectors1(items, 1)
    return np.concatenate([title_features, description_features], axis=1)

In [79]:
train_vec=word_vectors2(X_train)
test_vec=word_vectors2(X_test)

21000it [00:05, 3577.09it/s]
21000it [00:34, 611.20it/s]
9000it [00:02, 3853.62it/s]
9000it [00:13, 650.87it/s]


In [ ]:
lr_model=linear_model.LogisticRegression()
lr_model=lr_model.fit(train_vec, y_train)
metrics.accuracy_score(lr_model.predict(test_vec), y_test)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [ ]:
SVM_classifier = SVC(C=0.01, kernel='linear')
svc_model = SVM_classifier.fit(train_vec, y_train)
accuracy_score(svc_model.predict(test_vec), y_test)

### Что дальше? (6 баллов)

Для получения максимальной оценки вам нужно решить любые 2 пункта. Решение каждого пункта даст вам полтора балла:

1. Реализовать N-Gram модели текстовой классификации (__1.5 балла__)

2. Поработать с другими эмбеддингами для слов (например word2vec или GloVe) (__1.5 балла__)

3. Другие способы токенизации (pymorphy2, spaCy) (__1.5 балла__)

4. Добиться качества > 0.765 на тестовых данных (попробуйте другие токенизаторы, предобработку текста, и любые другие идеи, которые вам придут в голову) (__1.5 балла__)

Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

### Качество > 0.765 достигнуто с первым алгоритмом
### Способ токенизации pymorphy
Работаем так же, как с Mystem, но на вход принимаются только слова, поэтому предварительно обрабатываем Tweet Tokenizer.

In [ ]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()
from nltk.tokenize import TweetTokenizer


def lemma_py(sentence):
    tw = TweetTokenizer()
    sentence=tw.tokenize(sentence)
    for i in range (len(sentence)):
        result= pymorphy2_analyzer.parse(sentence[i])
        sentence[i]=result[0].normal_form
        i+=1
    return sentence
pytokens = {}
for i in senlist:
    freq = lemma_py(text)
    for token in freq:
        if token not in pytokens.keys():
            pytokens[token] = 1
        else:
            pytokens[token] += 1
pytokens=sorted(pytokens.items(), key=lambda x:x[1], reverse=True)[:10000]   
py_most_common_words = [x for x, y in pytokens]
py_word_to_index = {word: index for index, word in enumerate(py_most_common_words)}

In [ ]:
def py_text_to_bow(text: str) -> np.array:
    sentence_tokens = lemma_py(text)
    sent_vec = np.zeros(len(py_most_common_words))
    for token in sentence_tokens:
        if token in py_word_to_index:
            sent_vec[py_word_to_index[token]] += 1
    return sent_vec
def py_items_to_bow(items: np.array) -> np.array:
    modified = items[:,0] + " " + items[:, 1]
    vectors = np.zeros((len(items), len(py_most_common_words)))
    for i, desc in enumerate(modified):
        vectors[i] = py_text_to_bow(desc)
    return vectors
X_train_py = py_items_to_bow(X_train)
X_test_py = py_items_to_bow(X_test)

In [ ]:
from sklearn import linear_model
lr = linear_model.LogisticRegression()
lr_model = lr.fit(X_train_py, y_train)

print(accuracy_score(lr_model.predict(X_test_py), y_test))

from sklearn.svm import SVC
SVM_classifier = SVC(C=0.01, kernel='linear')
svc_model = SVM_classifier.fit(X_train_py, y_train)

print(accuracy_score(svc_model.predict(X_test_py), y_test))

### N-граммы. 
Берем биграммы и ищем самые популярные по тексту. Так же создаем словарь самых часто встречающихся и возвращаем вектор, где для каждой биграммы из словаря указано количество употреблений.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from nltk import ngrams
def ngrammodel(items: np.array, n: int) -> list:
    return[list(ngrams(i.split(),n)) for i in items]
X_train_n= ngrammodel(train_stem, 2)
X_test_n= ngrammodel(test_stem, 2)

In [ ]:
ngram_n={}
for item in X_train_n:
    for i in item:
        if i not in ngram_n:
            ngram_n[i]=1
        else:
            ngram_n[i]+=1
most_common_ngram={}
for(key, value) in sorted(ngram_n.items(), key=lambda x: x[1])[-10000:]:
    most_common_ngram[key]=value
most_common_ngram


                       

In [ ]:
def n_items_to_bow(items:np.array) -> np.array:
    def n_str_to_bow(text:str) -> np.array:
        d=dict(zip(most_common_ngram.keys(), np.zeros(len(most_common_ngram))))
        text=word_tokenize(text)
        for word1, word2 in zip(text[:-1], text[1:]):
            if (word1, word2) in d.keys():
                d[(word1, word2)]+=1
        return list(d.values())
    bow=[]
    for i in items:
        i = n_str_to_bow(i)
        bow.append(i)
    return np.array(bow)                                        

In [ ]:
X_train_nn = n_items_to_bow(train_stem)
X_test_nn = n_items_to_bow(test_stem)

In [ ]:
from sklearn import linear_model
lr = linear_model.LogisticRegression()
lr_model = lr.fit(X_train_nn, y_train)

print(accuracy_score(lr_model.predict(X_test_nn), y_test))

from sklearn.svm import SVC
SVM_classifier = SVC(C=0.01, kernel='linear')
svc_model = SVM_classifier.fit(X_train_nn, y_train)

print(accuracy_score(svc_model.predict(X_test_nn), y_test))

### Работаем с эмбеддингом Word2Vec
Смысл такой же, как с Fasttext, но в итоге accuracy выше.

In [ ]:
model=FastText.load_fasttext_format('word2vec-ruscorpora-300')
keys=[i.split('_')[0] for i in list(model.vocab.keys())]
model=dict(zip(keys, model.vectors))

train_wordvec=word_vectors2(train_stem)
test_wordvec=word_vectors2(test_stem)

lr_model=LogisticRegression()
lr_model=lr_model.fit(train_wordvec, y_train)
print(accuracy_score(lr_model.predict(test_wordvec), y_test))

In [ ]:
SVM_classifier = SVC(C=0.01, kernel='linear')
svc_model = SVM_classifier.fit(train_wordvec, y_train)
accuracy_score(svc_model.predict(test_wordvec), y_test)